In [86]:
import pandas as pd
import requests
import json

from api_luts import *

In [87]:
# how many zip codes?
len(ak_zip_codes)

273

In [88]:
# we can only request 100 zipcodes at once, so we need to break this list into parts
zipcode_chunks = [ak_zip_codes[0:100], ak_zip_codes[101:200], ak_zip_codes[201:]]

In [89]:
headers = {"apikey": "bdf542c0-f06e-11ee-869e-7ff58c93f49b"}

places = []

for zipcode_chunk in zipcode_chunks:
    zipcode_str = (",").join(zipcode_chunk)
    
    params = (("codes", zipcode_str),("country", "us"))
    response = requests.get('https://app.zipcodebase.com/api/v1/search', headers=headers, params=params)
    if response.status_code != 200:
        print("No response!")
    else:
        r_json = response.json()
        if len(r_json["results"].keys()) > 0:
            for zipcode in r_json["results"].keys():
                for place in r_json["results"][zipcode]:
                    city = place["city"]
                    places.append(city)
        else:
            print(f"No results for zip code string: {zipcode_str}!")

unique_places = list(set(places))
print(f"Requested location names for {len(ak_zip_codes)} Alaska zip codes...")
print(f"Returned {len(places)} location names...")
print(f"Returned {len(unique_places)} unique location names...")

Requested location names for 273 Alaska zip codes...
Returned 271 location names...
Returned 226 unique location names...


In [90]:
place_zipcode_dict = {}

for place in unique_places:
    params = (
   ("city",place),
   ("state_name","Alaska"),
   ("country","us"),)

    response = requests.get('https://app.zipcodebase.com/api/v1/code/city', headers=headers, params=params)
    if response.status_code != 200:
        print("No response!")
    else:
        r_json = response.json()
        #print(json.dumps(r_json, indent=3))
        if len(r_json["results"]) > 0:
            place_zipcode_dict[place] = r_json["results"]
        else:
            print(f"No results for {place}!")

In [91]:
place_zipcode_dict

{'Delta Junction': ['99737'],
 'South Naknek': ['99670'],
 'Sand Point': ['99661'],
 'Chignik': ['99564'],
 'Kalskag': ['99607'],
 'Coffman Cove': ['99918'],
 'Nome': ['99762'],
 'Juneau': ['99801', '99802', '99803', '99811', '99812', '99850'],
 'Lower Kalskag': ['99626'],
 'Shageluk': ['99665'],
 'Hope': ['99605'],
 'Pelican': ['99832'],
 'Kotlik': ['99620'],
 'Iliamna': ['99606'],
 'Eek': ['99578'],
 'Soldotna': ['99669'],
 'Auke Bay': ['99821'],
 'Homer': ['99603'],
 'Anvik': ['99558'],
 'Kasilof': ['99610'],
 'Two Rivers': ['99716'],
 'Pilot Station': ['99650'],
 'Stevens Village': ['99774'],
 'Saint Marys': ['99658'],
 'Wrangell': ['99929'],
 'Northway': ['99764'],
 'Deering': ['99736'],
 'Arctic Village': ['99722'],
 'Shishmaref': ['99772'],
 'Teller': ['99778'],
 'Hoonah': ['99829'],
 'Eagle River': ['99577'],
 'North Pole': ['99705'],
 'Chugiak': ['99567'],
 'Ouzinkie': ['99644'],
 'Denali National Park': ['99755'],
 'Ruby': ['99768'],
 'Mc Grath': ['99627'],
 'Noorvik': ['9976

In [92]:
# one strange return.... what is 31120? Google says its in Georgia?? I emailed the zipcodebase tech support to see if this is a bug
place_zipcode_dict["Tok"]

['99780', '31120', '99779']

In [93]:
# let's double check the returned zip codes against our original list to see if there are any discrepancies

returned_zipcodes = []
for place in place_zipcode_dict.keys():
    returned_zipcodes.extend(place_zipcode_dict[place])
returned_zipcodes = list(set(returned_zipcodes))


In [94]:
# we know from above that there should be two AK zip codes that did not return a place name in the API call
# These zip codes are 99636 for New Stuyahok and 99750 for Kivalina. We may just want to add these to our dictionary manually.
set(ak_zip_codes).difference(returned_zipcodes)

{'99636', '99750'}

In [95]:
# there are also some zip codes found in the API that are not part of our original zip code list
# these are the mystery Georgia zip code, plus 99512 which might be Anchorage and 99779 which is Tetlin
# we would want to remove 31120 but keep the other two
set(returned_zipcodes).difference(ak_zip_codes)

{'31120', '99512', '99779'}